# FUNCTIONS

In [1]:
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    
    
from nltk.corpus            import wordnet
from nltk.corpus            import stopwords
from nltk.stem.wordnet      import WordNetLemmatizer
import json
import re

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

try:
    import emoji
except:
    !pip install emoji
    
    import emoji
    
!pip install tweet-preprocessor
    
!pip install openpyxl

  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434674 sha256=58f8366e3a7fa9024f60bf3bd769ce07acd1997c2a65c2c372b936eaaff2ed66
  Stored in directory: c:\users\aman aryan\appdata\local\pip\cache\wheels\45\6c\46\a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk


[nltk_data] Downloading package stopwords to C:\Users\Aman
[nltk_data]     Aryan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Aman Aryan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to C:\Users\Aman
[nltk_data]     Aryan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49714 sha256=71c8e181c91896654546aec1c5c2dea86f596cab222acffd410472a1d3df1121
  Stored in directory: c:\users\aman aryan\appdata\local\pip\cache\wheels\4e\bf\6b\2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8913 sha256=da7e809717d4299c809c12831070102b1c3a321275e89eed9cb175cb0e8696f2
  Stored in directory: c:\users\aman aryan\appdata\local\pip\cache\wheels\e2\bd\55\048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built et-xmlfile


In [2]:
def nl_read_file(file_location):
    if(file_location.endswith('csv')):
        return pd.read_csv( file_location , engine = 'python')
    elif (file_location.endswith('tsv')):
        return pd.read_csv( file_location , engine = 'python' ,sep = '\t')
    elif (file_location.endswith('json')):
        return json.load( open(file_location,'r',encoding = 'utf8'))  

In [3]:
def string_preprocess(string, stop_words):
    string        = string.lower()
    string        = string.replace("’", "'")
    string        = string.replace("n\'t", " not")
    string        = string.replace("'re" , "")
    string        = string.replace("'ve" , "")
    string        = string.replace("'s"  , "")
    string        = string.replace("'m"  , "")
    string        = string.replace("'ll" , "")
    string        = string.replace("'ve" , "")
    string        = re.sub(r'[^\w\s\\d]' , " " , string)
    string        = re.sub(r'[\d]' , ' ' , string)
    string        = string.split(' ')
    tokens        = [token for token in string if (len(token) > 2) and (token not in stop_words) ]
    return tokens

def string_list_preprocess(string_list, stopwords):
    return [ string_preprocess(string, stop_words) for string in string_list]

In [4]:
def get_wordnet_pos(tag):
    if tag == 'J':
        return 'a'
    elif tag == 'R':
        return 'r'
    elif tag == 'V':
        return 'v'
    else:
        return 'n'

def string_lemmatize(tokens , lem):
    temp_list = ''
    tagged_words = nltk.pos_tag(tokens)
    for word in tagged_words:
        x =  lem.lemmatize( word[0], get_wordnet_pos( word[1][0] ) )
        temp_list = temp_list + ' ' + x
    return temp_list

def string_list_lemmatize(tokens_list):
    lemm_list = []
    lem = WordNetLemmatizer()
    
    for tokens in tokens_list:
        temp_list = string_lemmatize(tokens, lem)
        lemm_list.append(temp_list)
    return lemm_list

In [5]:
def convert_to_boolean(df):
    map_value = { 'n' : 0 , 'y' : 1 }
    for trait in traits:
        df[trait] = df[trait].map(map_value)
    return df

In [6]:
def preprocess_driver(file_location, stop_words):
    in_data                = nl_read_file(file_location)
    in_strings             = in_data['TEXT']
    out_strings            = string_list_preprocess(in_strings , stop_words)
    out_lemmatized_strings = string_list_lemmatize(out_strings)
    in_data['TEXT']        = out_lemmatized_strings
    return in_data

In [7]:
emoji_dict = json.load( open('input_files/emoji_dict.json','r',encoding = 'utf8'))  
def replace_emoji_text(text):
    for x in emoji_dict.keys():
        text = text.replace(x,emoji_dict[x])
    return text


nrc_lex = pd.read_excel (r'input_files/nrc_emotion_lex.xlsx', engine='openpyxl')
nrc_en_lex = list(nrc_lex['English (en)'])

def match_text( tweet_txt ):
    tweet_list = tweet_txt.split()
    intersects = list(set(tweet_list) & set(nrc_en_lex))
    if not intersects:
        return 0
    else:
        return 1

In [8]:
def preprocess_tweets( tweets ):
    
    import preprocessor as p
    p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.NUMBER)
    lem = WordNetLemmatizer()
    
    tweets_cleaned = {}

    for person in tweets.keys():

        text_dict = ''
        for dates in tweets[person].keys():
            
            string = emoji.demojize(  tweets[person][dates] )            
            string = replace_emoji_text(string)
            string = p.clean( string )
            string = string.replace('#','')
            string = string.replace('_',' ')
            string = string_preprocess( string , stop_words)
            string = string_lemmatize(string , lem)

            if( match_text(string) and len(string.split()) > 5):
                text_dict = text_dict + '.' + string

        tweets_cleaned[person] = text_dict[2:]
        
    return tweets_cleaned

# ESSAY DATA PROCESSING

In [9]:
stop_words = set(stopwords.words("english"))  #- set(['not','some'])
traits     = ['cEXT' , 'cNEU' , 'cAGR' , 'cCON' , 'cOPN' ]

In [10]:
df      = preprocess_driver( 'input_files/essays.csv', stop_words)
out_df  = convert_to_boolean( df )
out_df.to_csv( 'processed_datasets/essays.csv', index = False )

# TWITTER DATA PROCESSING

In [11]:
in_tweets_pre        = 'extract/tweets_pre_covid.json'
in_tweets_post       = 'extract/tweets_post_covid.json'
output_tweet_loc     = 'processed_datasets/tweets_processed.csv'

In [12]:
tweets_pre = nl_read_file(in_tweets_pre)
pre_tweets = preprocess_tweets( tweets_pre )

tweets_post = nl_read_file(in_tweets_post)
post_tweets = preprocess_tweets( tweets_post )

In [13]:
ids = []
pre_list = []
post_list = []
for person in pre_tweets.keys():
    ids.append(person)
    pre_list.append( pre_tweets[person] )
    post_list.append( post_tweets[person] )

df = pd.DataFrame()
df['PERSON'] = ids
df['PRETEXT'] = pre_list
df['POSTTEXT'] = post_list

df.to_csv(output_tweet_loc , index = False)